### RLHF Demo
Run post training on a pretrained GPT-2 model to understand RLHF. Steps will be SFT -> train reward model -> run grpo on pretrained llm on reward model. Rather than using TRL, I will be implementing grpo myself. Implementation will start with single gpu and then scaled to distributed system.

In [170]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

model.eval()

prompt = "The usual weather in California is"
inputs = tokenizer.encode(prompt, return_tensors='pt')

with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=1000,
        num_return_sequences=1,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


The usual weather in California is a bit of a mess.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still thick.

The sun is shining, but the clouds are still t

### Supervised fine tuning

What I need to do:
1. Preprocess data into chat template with EOS token. Ensure data is padded and make sure batches are truncated to fit context length.
2. Iterate through every batch and for each one calculate the loss (ONLY on the last assistant completion so the model learns prompt prediction). We use cross entropy btw.
3. Run a number of epochs on it.
4. Keep single threaded till we implement grpo as well.

In [183]:
from datasets import load_dataset, load_dataset_builder, get_dataset_split_names
from torch.utils.data import DataLoader
from pprint import pprint
import copy

# ---------------
# hyperparameters / config
num_epochs = 5
batch_size = 2
lr = 5e-5
weight_decay = 0.01
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
# ---------------

# create dataset train/val/test splits
train_sft_dataset = load_dataset("HuggingFaceH4/ultrachat_200k", split='train_sft').select(range(1000))
train_split_size = int(0.9 * len(train_sft_dataset))
train_split = train_sft_dataset.select(range(train_split_size))
val_split = train_sft_dataset.select(range(train_split_size, len(train_sft_dataset)))

# create chat template for tokenizer to use, gpt2 uses eos token so we need to add that as well
tokenizer.chat_template = """
{%- for message in messages %}
    {{- '<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n' }}
{%- endfor %}
{%- if add_generation_prompt %}
    {{- '<|im_start|>assistant\n' }}
{%- else %}
    {{- eos_token }}
{%- endif %}
"""

# preprocess data and create dataloader
def truncate_prompt(prompt):
    max_context_length = model.config.max_position_embeddings
    # while True:
    #     enc_chat_tem_ex


ending_msg_token_len = len(tokenizer.encode('<|im_end|>\n'))
def add_chat_tem(example):
    # truncate conversations by message boundary
    # truncation strategy:
    # 1. remove entire user/assistant pairs until it fits context window in fifo order but keep system prompt
    # 2. if you have one left, and it still isn't fitting into the context window, remove the tokens till it does
    example['og_messages'] = copy.deepcopy(example['messages'])
    max_context_length = model.config.max_position_embeddings
    has_system = 1 if example['messages'][0]['role'] == 'system' else 0
    enc_chat_tem_ex = tokenizer.apply_chat_template(example['messages'], tokenize=True, add_special_tokens=False)
    while True:
        diff = len(enc_chat_tem_ex) - max_context_length
        if diff <= 0:
            example['exclude'] = False
            break
        elif len(example['messages']) // 2 <= 1:
            example['exclude'] = True
            break

        del example['messages'][0 + has_system]
        del example['messages'][0 + has_system]


    # convert to chat template and keep track of # of tokens in last generation
    enc_chat_tem_ex = tokenizer.apply_chat_template(example['messages'], tokenize=True, add_special_tokens=False)
    example['input_ids'] = enc_chat_tem_ex
    end_size = (len(tokenizer.encode(example['messages'][-1]['content'], add_special_tokens=False)) + ending_msg_token_len)
    last_gen_start_ind = len(enc_chat_tem_ex) - end_size
    example['last_gen_start_ind'] = last_gen_start_ind
    return example

exclude_filter = lambda x: x['exclude']
train_split = train_split.map(add_chat_tem).filter(exclude_filter)
val_split = val_split.map(add_chat_tem).filter(exclude_filter)

# create custom collator for sft
class DataCollatorForSFT(DataCollatorForLanguageModeling):
    def __call__(self, features, return_tensors=None):
        last_gen_start_inds = [example['last_gen_start_ind'] for example in features]
        features = [{'input_ids': example['input_ids']} for example in features]
        batch = super().__call__(features, return_tensors=return_tensors)
        # scrappy but just assume we're calling with return_tensors='pt'
        batch['last_gen_start_inds'] = torch.tensor(last_gen_start_inds)

        return batch


data_collator = DataCollatorForSFT(
    tokenizer=tokenizer,
    mlm=False,
    return_tensors='pt'
)

train_dataloader = DataLoader(
    train_split,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator
)

Filter: 100%|██████████| 100/100 [00:00<00:00, 6477.49 examples/s]


In [184]:
pprint(train_split[0])

{'exclude': True,
 'input_ids': [27,
               91,
               320,
               62,
               9688,
               91,
               29,
               7220,
               198,
               40,
               460,
               470,
               4043,
               284,
               766,
               326,
               4017,
               23151,
               379,
               262,
               31096,
               12958,
               358,
               9594,
               13,
               314,
               4240,
               611,
               340,
               338,
               1103,
               393,
               655,
               257,
               30069,
               13,
               15467,
               835,
               11,
               340,
               338,
               5421,
               284,
               307,
               257,
               13899,
               2378,
               284,
          

In [173]:
'''
BxT -> BxTxP
A couple of issues I need to work out:
1. Padding inputs_ids in each prompt in the match correctly and ensuring attention mask is right
2. After we feed the prompt, we need to remove everything but the corresponding amount of tokens from the end of the prompt corresponding to the last assistant generation. We need to calculate labels that way simimlarly.
3. Calculate the loss for the sliced part (but each batch inner matrix is different size so need to think about that).
4. Backprop on that to train the model.
'''

# training run
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        input_ids = batch['input_ids']
        labels = batch['labels']
        last_gen_start_inds = batch['last_gen_start_inds']
        attention_mask = batch['attention_mask']

        print("INPUT IDS SHAPE:")
        print(input_ids.shape)
        print("MAX:", input_ids.max())
        print("MIN:", input_ids.min())

        # Debug tokenizer
        print(f"Tokenizer class: {type(tokenizer)}")
        print(f"Tokenizer vocab size: {len(tokenizer)}")
        print(f"Pad token: {tokenizer.pad_token}")
        print(f"Pad token ID: {tokenizer.pad_token_id}")
        print(f"EOS token ID: {tokenizer.eos_token_id}")
        print(f"Max position embeddings: {model.config.max_position_embeddings}")
        print(f"Model context window: {getattr(model.config, 'n_positions', 'Not specified')}")

        print("")

        # Check if pad token was added correctly
        if tokenizer.pad_token_id == 50256:  # This should match your max token
            print("Pad token ID matches max token - this is likely the issue!")

        # print("shape:", input_ids.shape)
        #
        # print("\n\n\n\n")
        #
        # first_prompt_input_ids = batch['input_ids'][0]
        # print(len(first_prompt_input_ids))
        # first_prompt_last_gen_token_len = batch['last_gen_start_inds'][0].item()
        # print(first_prompt_last_gen_token_len)
        #
        # print("\n\n\n\n")
        # print(tokenizer.decode(first_prompt_input_ids[:first_prompt_last_gen_token_len]))
        #
        #
        # mask labels that aren't included in last gen
        mask = torch.arange(input_ids.shape[1]) < last_gen_start_inds[:, None]
        # print("\n\n\n\n")
        # print(mask)
        labels[mask] = -100
        # print(labels)

        # run forward pass and calc loss
        model_outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        # print(model_outputs.logits.shape)


        break
    break

INPUT IDS SHAPE:
torch.Size([2, 1130])
MAX: tensor(50256)
MIN: tensor(11)
Tokenizer class: <class 'transformers.models.gpt2.tokenization_gpt2.GPT2Tokenizer'>
Tokenizer vocab size: 50257
Pad token: <|endoftext|>
Pad token ID: 50256
EOS token ID: 50256
Max position embeddings: 1024
Model max length: 20
Model context window: 1024
Pad token ID matches max token - this is likely the issue!


IndexError: index out of range in self

### Create the reward model

In [10]:
# The reward model generates a single logit representing the probability of that response (we use Bradley-Terry model of preferences)
class RewardModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, input_size * 4)
        self.fc2 = nn.Linear(input_size * 4, 1)

'''
Preference training loop steps:
1. Iterate through synthetic data of preferences.
2. Use the final hidden state last embedding as input to reward network.
3. Do that for both (otherwise you can use synthetic data and feed into the network the prompt + the response)
4. Use that to get the embeddings for both (just do one pass)
5. Calculate the loss based on Bradley-Terry and do backprop on the reward model network (you can mean the sums so you do it batchwise)

The idea we'll use is take the prompt reward string to get rewrd but only train on prompt to avoid mismatch problem.
'''


"\nPreference training loop steps:\n1. Iterate through synthetic data of preferences.\n2. Use the final hidden state last embedding as input to reward network.\n3. Do that for both (otherwise you can use synthetic data and feed into the network the prompt + the response)\n4. Use that to get the embeddings for both (just do one pass)\n5. Calculate the loss based on Bradley-Terry and do backprop on the reward model network (you can mean the sums so you do it batchwise)\n\nThe idea we'll use is take the prompt reward string to get rewrd but only train on prompt to avoid mismatch problem.\n"

In [16]:
from datasets import load_dataset

train_data = load_dataset('HuggingFaceH4/no_robots')['train']
train_data[0]

{'prompt': 'Please summarize the goals for scientists in this text:\n\nWithin three days, the intertwined cup nest of grasses was complete, featuring a canopy of overhanging grasses to conceal it. And decades later, it served as Rinkert’s portal to the past inside the California Academy of Sciences. Information gleaned from such nests, woven long ago from species in plant communities called transitional habitat, could help restore the shoreline in the future. Transitional habitat has nearly disappeared from the San Francisco Bay, and scientists need a clearer picture of its original species composition—which was never properly documented. With that insight, conservation research groups like the San Francisco Bay Bird Observatory can help guide best practices when restoring the native habitat that has long served as critical refuge for imperiled birds and animals as adjacent marshes flood more with rising sea levels. “We can’t ask restoration ecologists to plant nonnative species or to 